### decompose

- 5.30 shot加入thought
- 6.5晚tabfact_query 测试去掉lieral


In [1]:
import sys
sys.path.append('/media/disk1/chatgpt/zh/tabular_data')

In [2]:
import pandas as pd
from utils import parse_specific_composition, add_row_number, parse_specific_composition_zh
from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
from data_loader import TableFormat, TableLoader
from langchain.memory import ChatMessageHistory
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from sqlalchemy import create_engine
from executor import SQLManager
import sqlparse
embeddings = HuggingFaceBgeEmbeddings(
            model_name='BAAI/bge-large-en',
            model_kwargs={'device': 'cuda:3', 'trust_remote_code': True},
            encode_kwargs={'normalize_embeddings': True})

In [3]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
new_query_examples = [
                      "How many clubs play for the wru division one east in total?; How many clubs play 22 game for the wru division one east?",
                      "When did polona hercog partner with alberta brianti?; When did polona hercog partner with stephanie vogt?",
                      "what's the position gene hartley finish in 1952?; what's the position gene hartley finish in 1953?",
                      ]
num_k = 3
inds = [1, 5, 26]
table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
normalised_data = [table_loader.normalize_table(table_loader.dataset[inds[i]]) for i in range(num_k)]
example_samples = [TableFormat(format='none', data=normalised_data[i], save_embedding=False).get_sample_data(sample_type='random') for i in range(num_k)]
examples = [TableFormat.format_html(example_samples[i], normalised_data[i]['table']['caption']) for i in range(num_k)]

examples_prompt = PromptTemplate(input_variables=["query", "table", "new_query"], template=
"""
Table: {table}
Query: {query}
Decompose query: {new_query}""")

examples_dict = [{"query": normalised_data[i]['query'],
                                    "table": examples[i],
                                    "new_query": new_query_examples[i]} for i in range(num_k)]
decompose_prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""You are capable of converting complex query into sub queries. Based on the table, decompose original query into at most 2 complete sub queries which can solve original query. Output new query directly.""",
    suffix=
    """
Table: {table}
Query: {query}
Decompose query: """,
    input_variables=["query", "table"],
)




In [4]:
task_name = 'tabfact'
split = 'test'
model_name = 'gpt-3.5-turbo-0125'
# model = ChatOpenAI(model_name=model_name, openai_api_base="https://api.chatanywhere.com.cn/v1",
#                        openai_api_key="sk-WZtqZEeuE0Xb6syVghDgAxdwe0ASWLkQRGxl61UI7B9RqNC4", temperature=0.7).bind(logprobs=True)
schema_information = pd.read_csv(f"../result/aug/{task_name}_{split}_schema.csv", index_col='table_id')
aug_information = pd.read_csv(f"../result/aug/{task_name}_{split}_summary.csv", index_col='table_id')
composition_information = pd.read_csv(f"../result/aug/{task_name}_{split}_composition.csv", index_col='table_id')
# string_information = pd.read_csv(f"../result/aug/{task_name}_{split}_string.csv", index_col='table_id')
engine = create_engine('sqlite:////media/disk1/chatgpt/zh/tabular_data/db/sqlite/tabfact.db', echo=False)
manager = SQLManager(engine=engine)

In [5]:
table_loader = TableLoader(table_name=task_name, split='test', use_sample=False, small_test=False)
sample = table_loader.normalize_table(table_loader.dataset[2])

### step-back

In [6]:
from langchain.prompts.prompt import PromptTemplate
from langchain.prompts.few_shot import FewShotPromptTemplate
from data_loader import TableFormat
inds = [8, 19, 33,]
num_k = 3
table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
normalised_data = [table_loader.normalize_table(table_loader.dataset[inds[i]]) for i in range(num_k)]
example_samples = [TableFormat(format='none', data=normalised_data[i], save_embedding=False).get_sample_data(sample_type='random') for i in range(num_k)]
examples = [TableFormat.format_html(example_samples[i], normalised_data[i]['table']['caption']) for i in range(num_k)]
new_query_examples = [
    # "Which country uses the US dollar as its currency and has the Federal Reserve as its central bank?",
    "which college list be public?",
    "what is the constuctor of the jean behra?",
    "what are all the points when jason richardson be their leading scorer that month?"
    # "how many race did davíd garza pérez participate in each season?"
    ]
examples_prompt = PromptTemplate(input_variables=["query", "new_query"], template=
"""
Table: {table}
Query: {query}
New query: {new_query}""")

examples_dict = [{"query": normalised_data[i]['query'],
                  "table": examples[i],
                    "new_query": new_query_examples[i]} for i in range(num_k)]
step_back_prompt = FewShotPromptTemplate(
    examples=examples_dict,
    example_prompt=examples_prompt,
    prefix="""Based on the table, your task is to step back and paraphrase a question to a more generic step-back question, which is easier to answer.""",
    suffix=
    """
Table: {table}
Query: {query}
New query:""",
    input_variables=["query", "table"],
)

In [7]:
def get_k_shot_with_answer_noinfo(k: int=1):
    sqls = ["SELECT MIN(points) FROM DF WHERE rider = 'roger dutton / tony wright';"
            ]
    thoughts = ["Based on the SQL query provided, the minimum number of points that Roger Dutton / Tony Wright received in the 1972 Isle of Man TT event was 3. 3 is the fewest points they received. "]
    tables = ["<table>\n<caption>1972 isle of man tt</caption>\n<thead>\n<tr><th>  MIN(points)</th></tr>\n</thead>\n<tbody>\n<tr><td>3            </td></tr>\n</tbody>\n</table>"]
    claims = ["2 be the fewest point that roger dutton / tony wright receive"]
    # inds from test split
    examples_prompt = PromptTemplate(input_variables=["SQL", "table", "claim", "thought", "output"], template=
    """
SQL Excuted: 
```{SQL}```
Sub-table: {table}
Query: {claim}
Thought: {thought}
Answer: {output}
    """)
    examples_dict = dict(zip(["SQL", "table", "claim", "thought", "output"], [sqls[0], tables[0], claims[0], thoughts[0], '0']))
    prompt_template = FewShotPromptTemplate(
        examples=[examples_dict],
        example_prompt=examples_prompt,
        prefix="""Below is a sub-table generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering. Think step by step and verify whether the provided claim/query is true or false.
You should output in the following format:
Thought: your step by step thought
Answer: return 0 if the query is false, or 1 if the query true
Below is an example.""",
        suffix=
        """
SQL Excuted: 
```{SQL}```
Sub-table: {table}
Query: {query}""",
        input_variables=["table", "query", "SQL"],
)
    return prompt_template

def get_k_shot_with_answer(k: int=1):
    sqls = ["SELECT DISTINCT Type FROM DF WHERE Type != 'audio';"
            ]
    thoughts = ["Based on the SQL query and the extra information provided, the types include audio or video. Therefore, other than audio, the payload type is video, the claim is True."]
    tables = ["<table>\n<thead>\n<tr><th> Type </th></tr>\n</thead>\n<tbody>\n<tr><td>video   </td></tr>\n<tr><td>audio/video   </td></tr>\n</tbody>\n</table>"]
    claims = ["other than audio, the payload type is video."]
    extras = ["The payload types for audio include audio, video, and audio/video."]
    # inds from test split
    examples_prompt = PromptTemplate(input_variables=["SQL", "table", "information", "claim", "thought", "output"], template=
    """
SQL Excuted: 
```{SQL}```
Sub-table: {table}
Extra information:
{information}

Claim: {claim}
Thought: {thought}
Answer: {output}
    """)
    examples_dict = dict(zip(["SQL", "table", "information", "claim", "thought", "output"], [sqls[0], tables[0],extras[0], claims[0], thoughts[0], '1']))
    prompt_template = FewShotPromptTemplate(
        examples=[examples_dict],
        example_prompt=examples_prompt,
        prefix="""Below is a sub-table generated by excuting the corresponding SQL with additional extra information. You need to understand the logic behind the SQL filtering. Think step by step and verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true.
You should output in the following format:
Thought: your step by step thought
Answer: final answer
Below is an example.""",
        suffix=
        """
SQL Excuted: 
```{SQL}```
Sub-table: {table}
Extra information:
{information}

Claim: {query}
""",
        input_variables=["table", "query", "SQL", "information"],
)
    return prompt_template


In [8]:
# def get_k_shot_with_answer(k: int=1):
#     sqls = ["SELECT MIN(points) FROM DF WHERE rider = 'roger dutton / tony wright';"
#             ]
#     thoughts = ["Based on the SQL query provided, the minimum number of points that Roger Dutton / Tony Wright received in the 1972 Isle of Man TT event was 3. 3 is the fewest points they received. "]
#     tables = ["<table>\n<caption>1972 isle of man tt</caption>\n<thead>\n<tr><th>  MIN(points)</th></tr>\n</thead>\n<tbody>\n<tr><td>3            </td></tr>\n</tbody>\n</table>"]
#     claims = ["2 be the fewest point that roger dutton / tony wright receive"]
#     # inds from test split
#     examples_prompt = PromptTemplate(input_variables=["SQL", "table", "claim", "thought", "output"], template=
#     """
# SQL Excuted: 
# ```{SQL}```
# Sub-table: {table}
# Thought: {thought}
# Query: {claim}
# Answer: {output}
#     """)
#     examples_dict = dict(zip(["SQL", "table", "claim", "thought", "output"], [sqls[0], tables[0], claims[0], thoughts[0], '0']))
#     prompt_template = FewShotPromptTemplate(
#         examples=[examples_dict],
#         example_prompt=examples_prompt,
#         prefix="""Below is a sub-table generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering. Think step by step and verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true.
# You should output in the following format:
# Thought: your step by step thought
# Answer: final answer
# Below is an example.""",
#         suffix=
#         """
# SQL Excuted: 
# ```{SQL}```
# Sub-table: {table}
# Extra information:
# {information}
# Query: {query}
# """,
#         input_variables=["table", "query", "SQL", "information"],
# )
#     return prompt_template

In [9]:
# table_loader = TableLoader(table_name='tabfact', split='validation', use_sample=True, small_test=False)
#     table_loader_wiki = TableLoader(table_name='wikitable', split='train', use_sample=True, small_test=False)
#     inds = [3, 6, 260, 33]
#     Output_examples = [
#                        'team, goals_for',
#                        'year, game, platform_s',
#                        'name, population_density_km_2_, population_2011_census_'
#                        'leading_scorer, score, date']
#     linking_examples = ['the team -> team, the most goal for -> goals_for',
#                         'gamecube -> platform_s, gamecube game -> game, the first 3 year -> year',
#                         'alberta -> name, population density -> population_density_km_2_, 4257744 less people -> population_2011_census_, 2011 -> population_2011_census_'
#                         'jason richardson -> leading_scorer, month -> date, 23 point per game -> score'
#     ]
#     examples_prompt = PromptTemplate(input_variables=["table", "claim", "output", "linking"], template=
#     """
#     Table: {table}
#     Query: {claim}
#     Column linking: {linking}
#     Columns: {output}""")
    # num_k = 2

In [10]:
def get_k_shot_with_aug(k: int=2):
    examples_prompt = PromptTemplate(input_variables=["table", "claim", "output", "linking"], template=
    """
    Table: {table}
    Query: {claim}
    Column linking: {linking}
    Columns: {output}""")
    examples_dict = []
    examples_dict.extend([{"table": '<table>\n<caption>Hoot Kloot</caption>\n<thead>\n<tr><th> Number</th><th> Title</th><th> Directed_by_</th><th> Released_</th></tr>\n</thead>\n</table>',
                                        "claim": 'Mesa Trouble was the last title that sid marcus directed.',
                                        "aug": "The table contains information about the Hoot Kloot animated series, including the episode number, title, director, and release year. \n1. Number: The episode number in the series \n2. Title: The title of the episode \n3. Directed_by_: The director of the episode \n4. Released_: The release year of the episode",
                                        "linking": "the last title -> Released_, the last title-> Number, title -> Title, sid marcus -> Directed_by_",
                                        "output": "Released_, Number, Title, Directed_by_"}])
    examples_dict.extend([{"table": '<table>\n<caption>1943–44 Chicago Black Hawks season</caption>\n<thead>\n<tr><th>  num</th><th>       Date</th><th>            Visitor</th><th>  Score</th><th>               Home</th><th>  Record</th></tr>\n</thead>\n</table>',
                                        "claim": 'The difference in score in the december 19th win was 3.',
                                        "aug": 'The table contains information about the 1943-44 Chicago Black Hawks season, including the date, visitor, score, home team, record, and points for each game. \n1. num: The game number in the season \n2. Date: The date of the game\n3. Vistor: The visiting team\n4. Score: The final score, with the visitor score listed first\n5. Home: The home team\n6. Record: The team win-loss-overtime loss record at the time of the game',
                                        "linking": 'difference in score -> Score, december 19th -> Date',
                                        "output": 'Date, Score'}])
    prompt_template = FewShotPromptTemplate(
        examples=examples_dict,
        example_prompt=examples_prompt,
        prefix=
        """
    Based on the Table below, your task is accurately output columns related to the query or contain useful information about the query. This process involves linking similar words or semantically similar terms to columns in the Table.
    Approach this task as follows:
    Read the query thoroughly and list every possible link from query term to column in the Table. 
    Then Based on the column linking, output all useful columns at last. Make sure all columns in the link step are included and every column is in the Table.""",
        suffix=
        """
    Table: {table}
    Query: {claim}""",
        input_variables=["table", "claim"],
)
    return prompt_template




def get_k_shot_with_aug_wiki(k: int=2):
    table_loader = TableLoader(table_name='wikitable', split='train', use_sample=True, small_test=False)
    examples_dict = []
    examples_dict.extend([{"table": '<table>\n<caption>Hoot Kloot</caption>\n<thead>\n<tr><th> Number</th><th> Title</th><th> Directed_by_</th><th> Released_</th></tr>\n</thead>\n</table>',
                                        "claim": table_loader.dataset[95]['question'],
                                        "aug": "The table contains information about the Hoot Kloot animated series, including the episode number, title, director, and release year. \n1. Number: The episode number in the series \n2. Title: The title of the episode \n3. Directed_by_: The director of the episode \n4. Released_: The release year of the episode",
                                        "linking": "the last title -> Released_, the last title-> Number, title -> Title, sid marcus -> Directed_by_",
                                        "output": "Released_, Number, Title, Directed_by_"}])
    examples_dict.extend([{"table": '<table>\n<caption>1943–44 Chicago Black Hawks season</caption>\n<thead>\n<tr><th>  num</th><th>       Date</th><th>            Visitor</th><th>  Score</th><th>               Home</th><th>  Record</th></tr>\n</thead>\n</table>',
                                        "claim": 'what was the difference in score in the december 19th win?',
                                        "aug": 'The table contains information about the 1943-44 Chicago Black Hawks season, including the date, visitor, score, home team, record, and points for each game. \n1. num: The game number in the season \n2. Date: The date of the game\n3. Vistor: The visiting team\n4. Score: The final score, with the visitor score listed first\n5. Home: The home team\n6. Record: The team win-loss-overtime loss record at the time of the game',
                                        "linking": 'difference in score -> Score, december 19th -> Date',
                                        "output": 'Date, Score'}])
    
    # "The table contains information about the Hoot Kloot animated series, including the episode number, title, director, and release year."
    # "№<The episode number in the series>\nTitle<The title of the episode>\nDirected_by_<The director of the episode>\nReleased_<The release year of the episode>"
    examples_prompt = PromptTemplate(input_variables=["table", "claim", "output", "linking"], template=
    """
    Table: {table}
    Query: {claim}
    Column linking: {linking}
    Columns: {output}""")
    prompt_template = FewShotPromptTemplate(
        examples=examples_dict,
        example_prompt=examples_prompt,
        prefix=
        """
    Based on the Table below, your task is accurately output columns related to the query or contain useful information about the query. This process involves linking similar words or semantically similar terms to columns in the Table.
    Approach this task as follows:
    Read the query and extra information thoroughly and list every possible link from query term to column in the Table. 
    Then Based on the column linking, output all useful columns at last. Make sure all columns in the link step are included and every column is in the Table.""",
        suffix=
        """
    Table: {table}
    Query: {claim}""",
        input_variables=["table", "claim"],
)
    return prompt_template


In [11]:
from utils import parse_output
def scene_A(query, sample, k=0, verbose=True):
    row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""Our ultimate goal is to answer query based on the original table. Below we have a sub-table with rows randomly sampled from the original table, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the extra information, write a SQLITE3 SELECT SQL statement using table DF that complete query. Directly Output SQL, do not add other string.
sub-table: {table}
Query: {claim}
SQL: """)
    formatter = TableFormat(format='none', data=sample, save_embedding=True, embeddings=embeddings)
    
    formatter.normalize_schema(schema_information.loc[sample['table']['id']]['schema'])
    sample_data = formatter.get_sample_data(sample_type='embedding',k=k, query=query)
    with get_openai_callback() as cb:
        llm_chain = LLMChain(llm=model, prompt=get_k_shot_with_aug(), verbose=verbose)
        # summary_aug, column_aug = aug_information.loc[sample['table']['id']]['summary'], aug_information.loc[sample['table']['id']]['column_description'] 
        # col_names, col_infos = parse_output(column_aug, pattern=r'([^<]*)<([^>]*)>')
        # extra_col_info = []
        # for i_c in range(len(col_names)):
        #     extra_col_info.append(f'{i_c + 1}. {col_names[i_c]}: {col_infos[i_c]}')
        
        stage_1_batch_pred = llm_chain.batch([dict({'table': TableFormat.format_html(data=sample_data, table_caption=sample['table']['caption']),
                                            'claim': query,
                                            })], return_only_outputs=True)[0]['text']
        if verbose:
            print(stage_1_batch_pred)
        stage_1_batch_pred = stage_1_batch_pred.split(':')[-1]
        if verbose:
            print(stage_1_batch_pred)
        # extra_cols = formatter.get_sample_column(embeddings, column_aug)
        # stage 2: SQL generation
        llm_chain = LLMChain(llm=model, prompt=row_instruction, verbose=verbose)
        columns = list(set([c.strip() for c in stage_1_batch_pred.split(',')]))
        
        try: 
            # formatter.all_data = formatter.all_data.loc[:, columns]
            sample_data = add_row_number(sample_data.loc[:, columns])
        except:
            sample_data = add_row_number(sample_data)
        
        extra_information = []
        # tuples = parse_specific_composition_zh(composition_information.loc[sample['table']['id']]['composition'], sample_data.columns)
        # for col, com in tuples:
        #     if len(pd.unique(formatter.all_data[col])) < 6:
        #         com += f' (Values like {", ".join(list(formatter.all_data[col].dropna().unique().astype(str)))})'
        #         extra_information.append(col + ':' + com)
        #     else:
        #         com += f' (Values like {", ".join(list(formatter.all_data[col].dropna().unique()[:3].astype(str)))}...)'
        #         extra_information.append(col + ':' + com)
        # extra_information.append('row_number: row number in the original table')
        stage_2_batch_pred = llm_chain.batch([dict({'table': TableFormat.format_html(data = sample_data, table_caption=sample['table']['caption']),
                                            'claim': query,
                                            # 'aug':  summary_aug + '\nColumn information:\n' + '\n'.join(extra_information)
                                            })], return_only_outputs=True)[0]['text'].replace("–", "-").replace("—", "-").replace("―", "-").replace("−", "-")
        # print(stage_2_batch_pred)
    # stage 3: SQL Excution
    try: 
        execute_data = manager.execute_from_df(stage_2_batch_pred, add_row_number(formatter.all_data), table_name='DF')[:20]
    except:
        execute_data = formatter.all_data[:20]
        stage_2_batch_pred = 'SELECT * from DF;'
    if len(execute_data) == 0:
        return query, stage_2_batch_pred, 'No data from database', cb.total_tokens
    return query, stage_2_batch_pred, TableFormat.format_html(data=execute_data, table_caption=sample['table']['caption']), cb.total_tokens

### RUN

In [12]:
def eval_blury_string(pred_list):
    pred_label = []
    for pred in pred_list:
        predict_ans = pred.split('\n')[-1]
        if '0' in predict_ans:
            predict_ans = '0'
        elif '1' in predict_ans:
            predict_ans = '1'
        else:
            predict_ans = '2'
        pred_label.append(predict_ans)
    return pred_label

In [13]:
import concurrent.futures
from typing import List
import os
import json
from langchain_community.callbacks import get_openai_callback
def parallel_run(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = [executor.submit(func, arg) for arg in args_list]
        return [future.result() for future in concurrent.futures.as_completed(results)]

def parallel_run_kwargs(func, args_list):
    with concurrent.futures.ThreadPoolExecutor() as executor:
        results = executor.map(lambda kwargs: func(**kwargs), args_list)
        return list(results)

def save_csv(input_list: List[List], label_list: List, file_path):
    import pandas as pd
    directory = os.path.dirname(file_path)
    if not os.path.exists(directory):
        os.makedirs(directory)

    assert len(input_list) == len(label_list)
    df = pd.DataFrame()
    for i in range(len(label_list)):
        df[label_list[i]] = pd.Series(input_list[i])
    if os.path.exists(file_path) and file_path.endswith('.csv'):
        df_origin = pd.read_csv(file_path)
        df = pd.concat([df_origin, df], axis=0)
    df.to_csv(file_path, index=False, encoding='utf-8')

## 调整extrainformation的位置

In [14]:
from utils import parse_output
# Please provide a clear, complete statement in response to the question. However, if you cannot answer the query based on the sub-table or sub-table returns no data, just output 'Cannot get answer from sub-table'. 
answer_instruction = PromptTemplate(input_variables=["SQL", "table", "claim"], 
                                    template="""
Below is a sub-table generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering and answer the query using the final sub-table. 
SQL Excuted: 
```{SQL}```
Sub-table: 
{table}
Query: {claim}
Please provide a clear, complete statement in response to the query. If you cannot answer the query based on the sub-table, just return 'Cannot get answer from sub-table'.""" )
def scene_B(query, sample,k=0,  verbose=False):
    row_instruction = PromptTemplate(input_variables=["table", "claim", "aug"], 
                                 template="""Our ultimate goal is to answer query based on the original table. Below we have a sub-table with rows randomly sampled from the original table, you are required to infer the data distribution and format from the sample data of the sub-table. Carefully analyze the query, based on the extra information, write a SQLITE3 SELECT SQL statement using table DF that complete query. Directly Output SQL, do not add other string.

sub-table: {table}
Query: {claim}
SQL: """)
    
    formatter = TableFormat(format='none', data=sample, save_embedding=True, embeddings=embeddings)
    formatter.normalize_schema(schema_information.loc[sample['table']['id']]['schema'])
    sample_data = formatter.get_sample_data(sample_type='embedding', k=k,query=query)
    # get columns
    with get_openai_callback() as cb:
        llm_chain = LLMChain(llm=model, prompt=get_k_shot_with_aug_wiki(), verbose=verbose)
        stage_1_batch_pred = llm_chain.batch([dict({'table': TableFormat.format_html(data=sample_data, table_caption=sample['table']['caption']),
                                            'claim': query,
                                            })], return_only_outputs=True)[0]['text']
        stage_1_batch_pred = stage_1_batch_pred.split(':')[-1]
        
        # stage 2: SQL generation
        llm_chain = LLMChain(llm=model, prompt=row_instruction, verbose=verbose)
        columns = list(set([c.strip() for c in stage_1_batch_pred.split(',')]))
        
        try: 
            sample_data = add_row_number(sample_data.loc[:, columns])
        except:
            sample_data = add_row_number(sample_data)
        # extra_information = []
        # tuples = parse_specific_composition_zh(composition_information.loc[sample['table']['id']]['composition'], sample_data.columns)
        # for col, com in tuples:
        #     if len(pd.unique(formatter.all_data[col])) < 6:
        #         com += f' (Values like {", ".join(list(formatter.all_data[col].dropna().unique().astype(str)))})'
        #         extra_information.append(col + ':' + com)
        #     else:
        #         com += f' (Values like {", ".join(list(formatter.all_data[col].dropna().unique()[:3].astype(str)))}...)'
        #         extra_information.append(col + ':' + com)
        # extra_information.append('row_number: row number in the original table')
        stage_2_batch_pred = llm_chain.batch([dict({'table': TableFormat.format_html(data=sample_data, table_caption=sample['table']['caption']),
                                            'claim': query,
                                            # 'aug':  summary_aug + '\n Column information:' + '\n'.join(extra_information)
                                            })], return_only_outputs=True)[0]['text'].replace("–", "-").replace("—", "-").replace("―", "-").replace("−", "-")
    
        
        # stage 3: SQL Excution
        try: 
            execute_data= manager.execute_from_df(stage_2_batch_pred, add_row_number(formatter.all_data), table_name='DF')[:20]
        except:
            execute_data = formatter.all_data[:20]
            stage_2_batch_pred = 'SELECT * from DF;'
        llm_chain = LLMChain(llm=model, prompt=answer_instruction, verbose=verbose)
        response = llm_chain.batch([dict({'table': TableFormat.format_html(execute_data, table_caption=sample['table']['caption']),
                                                'claim': query,
                                                'SQL':  stage_2_batch_pred
                                                })], return_only_outputs=True)[0]['text']
    # print("total_tokens:", cb.total_tokens)
    return response, cb.total_tokens

In [15]:
from langchain_openai import AzureChatOpenAI

import os

os.environ["AZURE_OPENAI_API_KEY"] = "0c75de50975e4f278b882fe90da47f2f"
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://ces.openai.azure.com"
os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-35-turbo"
# os.environ["AZURE_OPENAI_API_KEY"] = "2b219db0d2984f9dae28b651ab8ab3d9"
# os.environ["AZURE_OPENAI_ENDPOINT"] = "https://smsh.openai.azure.com/"
# os.environ["AZURE_OPENAI_API_VERSION"] = "2024-02-01"
# os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-35-turbo"
model = AzureChatOpenAI(
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    azure_deployment=os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"],
    temperature=0.3,
    max_retries=8, request_timeout=600
)

### query aug

In [16]:
a

NameError: name 'a' is not defined

In [18]:
##### no info

from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
import datetime
# from FlagEmbedding import FlagReranker
from openai import BadRequestError, RateLimitError
from tqdm.notebook import tqdm
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
# model = ChatOpenAI(model_name='gpt-3.5-turbo-0125', openai_api_base="https://api.chatanywhere.com.cn/v1",
#                        openai_api_key="sk-WZtqZEeuE0Xb6syVghDgAxdwe0ASWLkQRGxl61UI7B9RqNC4", temperature=0.01)
# save_path = f"../result/final_answer/tabfact_noaug_0_query_{datetime.datetime.now().strftime('%m-%d_%H-%M-%S')}.csv"
save_path = f"../result/final_answer/tabfact_noaug_0_query_06-10_01-31-32.csv"

# reranker = FlagReranker('BAAI/bge-reranker-large', use_fp16=True)

muilti_answer_instruction = PromptTemplate(input_variables=["information", "claim"], 
# template="""You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
template = """
Below is a sub-table generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering. Complete task with the help of extra information below.

SQL Excuted: 
```{SQL}```
Sub-table:
{table}
Extra information:
{information}
Query: {query}

Task: verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true. Please think step by step and return 0/1 at last.
""" )
# Task: answer the last question given in the query. Only return the string instead of other format information. Do not repeat the question.
# Task: verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true. Please think step by step and return 0/1 at last.


# muilti_answer_instruction = get_k_shot_with_answer()
# for sample_n in range(3):
tokens = []
outputs = []
labels = []
ids = []
extra_quries = []
i = 1910
sample_k = 0
with tqdm(total=len(table_loader.dataset) - 1910, desc=f"Processing",ncols=150) as pbar:
    while i < len(table_loader.dataset):
        try:
            sample = table_loader.normalize_table(
                                table_loader.dataset[i])
            all_tokens = 0
            all_queries = []
            formatter = TableFormat(format='none', data=sample, save_embedding=False)
            sample_data = formatter.get_sample_data(sample_type='random',k=sample_k, query=sample['query'])
            with get_openai_callback() as cb:
                llm_chain = LLMChain(llm=model, prompt=step_back_prompt, verbose=False)
                batch_pred = llm_chain.batch([{"query": sample['query'], "table": TableFormat.format_html(sample_data)}], return_only_outputs=True)
                if batch_pred[0]['text'].strip() != sample['query']:
                    all_queries.append(batch_pred[0]['text'].strip())
                llm_chain = LLMChain(llm=model, prompt=decompose_prompt, verbose=False)
                batch_pred = llm_chain.batch([{"query": sample['query'], "table": TableFormat.format_html(sample_data)}], return_only_outputs=True)
                all_queries.extend([q.strip() for q in batch_pred[0]['text'].split(';')])
            all_tokens += cb.total_tokens
            all_queries = list(set(all_queries))
            args_list = [{"query": q, "sample": sample, "k": sample_k} for q in all_queries]
            # print(len(args_list))
            ans_from_B = parallel_run_kwargs(scene_B, args_list)
            results = [res[0] for res in ans_from_B if 'Cannot get answer from sub-table' not in res[0]]
            all_tokens += sum([res[1] for res in ans_from_B])
            #With answer
            # results= []
            with get_openai_callback() as cb:
                imp_input = scene_A(sample['query'], sample,sample_k,  False)
                llm_chain = LLMChain(llm=model, prompt=get_k_shot_with_answer(), verbose=False)
                batch_pred = llm_chain.batch([{"query":sample['query'],"SQL": imp_input[1], "table": imp_input[2], "information": '\n'.join(results)}], return_only_outputs=True)
                # print(batch_pred[0])
                all_tokens += cb.total_tokens
                # print('ALL TOKENS', all_tokens)
                ids.append(i)
                labels.append(sample['query'])
                outputs.append(batch_pred[0]['text'])
                tokens.append(all_tokens)
                extra_quries.append(';'.join(all_queries)) 
                if (i + 1) % 10 == 0:
                        print(f'saving {i}')
                        save_csv([outputs, labels, ids, tokens, extra_quries], ['preds', 'statements','ids', 'tokens', 'extra'], file_path=save_path)
                        outputs = []
                        labels = []
                        ids = []
                        tokens = []
                        extra_quries = []
                i += 1
                pbar.update(1)
        except ValueError as e:
            print(f'******************Value Error {i}****************************')
            i += 1
            pbar.update(1)
        
        except BadRequestError as e:
            print('*************************Bad Request**************')
            i += 1 
            print('*************************Rate limit**************')
            pass
        
        
# save_csv([outputs, labels, ids, tokens, extra_quries], ['preds', 'statements','ids', 'tokens', 'extra'], file_path=save_path)
    
#With no answer
# temp = [f"""
# SQL Excuted for extra information: 
# ```{res[1]}```
# Sub-table for extra information: {res[2]}""" for res in results if res[2] != 'No data from database']
# imp_input = scene_A(sample['query'], sample, False)
# llm_chain = LLMChain(llm=model, prompt=muilti_answer_instruction, verbose=True)
# batch_pred = llm_chain.batch([{"query": sample['query'],"SQL": imp_input[1], "table": imp_input[2], "information": '\n'.join(temp)}], return_only_outputs=True)
# print(batch_pred[0])
# outputs.append(batch_pred[0]['text'])


Processing:   0%|                                                                                             …

saving 1919
saving 1929
saving 1939
saving 1949
saving 1959
saving 1969
saving 1979
saving 1989


In [19]:
len(outputs)

8

In [20]:
save_csv([outputs, labels, ids, tokens, extra_quries], ['preds', 'statements','ids', 'tokens', 'extra'], file_path=save_path)

In [ ]:
print(all_queries)
outputs

['When did the witchfinder episode happen?', 'When did the sweet dream episode happen?', 'what is the order of the sweet dream episode in the series?']


['Thought: Since the sub-table does not contain any data, we cannot verify the claim based on the provided information.\nAnswer: 0']

In [ ]:
a
1. prompt是否添加verrify instruction
2. 

SyntaxError: invalid syntax (578050692.py, line 2)

In [ ]:
data = pd.read_csv('../result/answer/tabfact_test_query_gpt35_0125.csv')
data2 = pd.read_csv('../result/answer/tabfact_05-13_08-47-41.csv')
data3 = pd.read_csv('./result/answer/tabfact_05-14_13-38-48.csv')
data['final_answer'] = data['preds'].apply(lambda x: x.split('Answer:')[-1].strip())
data2['final_answer'] = data2['preds'].apply(lambda x: x.split('Answer:')[-1].strip())
data3['final_answer'] = data3['preds'].apply(lambda x: x.split('Answer:')[-1].strip())

In [ ]:
count = 0
for i in range(len(table_loader.dataset)):
    if i in list(data['ids']):
        i += 1
    else:
        count+= 1
        i += 1

In [21]:
##### add residual 

from langchain.chains import LLMChain
from langchain_openai import ChatOpenAI, OpenAI
import datetime
# from FlagEmbedding import FlagReranker
from openai import BadRequestError, RateLimitError
from tqdm.notebook import tqdm
table_loader = TableLoader(table_name='tabfact', split='test', use_sample=False, small_test=True)
model = ChatOpenAI(model_name='gpt-3.5-turbo-0125', openai_api_base="https://api.chatanywhere.tech/v1",
                       openai_api_key="sk-bLZSHx4pKfPRZkYyIyyvUHSEjrlqj5sh2QIsxOM23yJnyoGD", temperature=0.3)
save_path = f"../result/final_answer/tabfact_noaug_0_query_06-10_01-31-32.csv"

data = pd.read_csv(save_path)

muilti_answer_instruction = PromptTemplate(input_variables=["information", "claim"], 
# template="""You are a brilliant table executor with the capabilities information retrieval, table parsing, table partition and semantic understanding who can understand the structural information of the table.
template = """
Below is a sub-table generated by excuting the corresponding SQL. You need to understand the logic behind the SQL filtering. Complete task with the help of extra information below.

SQL Excuted: 
```{SQL}```
Sub-table:
{table}
Extra information:
{information}
Query: {query}

Task: verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true. Please think step by step and return 0/1 at last.
""" )
# Task: answer the last question given in the query. Only return the string instead of other format information. Do not repeat the question.
# Task: verify whether the provided claim/query is true or false, return 0 if it's false, or 1 if it's true. Please think step by step and return 0/1 at last.


# muilti_answer_instruction = get_k_shot_with_answer()
# for sample_n in range(3):
tokens = []
outputs = []
labels = []
ids = []
extra_quries = []
i = 0
sample_k = 0
with tqdm(total=len(table_loader.dataset), desc=f"Processing",ncols=150) as pbar:
    while i < len(table_loader.dataset):
        if i in list(data['ids']):
            i += 1
        else:
            try:
                sample = table_loader.normalize_table(
                                table_loader.dataset[i])
                all_tokens = 0
                all_queries = []
                formatter = TableFormat(format='none', data=sample, save_embedding=False)
                sample_data = formatter.get_sample_data(sample_type='random',k=sample_k, query=sample['query'])
                with get_openai_callback() as cb:
                    llm_chain = LLMChain(llm=model, prompt=step_back_prompt, verbose=False)
                    batch_pred = llm_chain.batch([{"query": sample['query'], "table": TableFormat.format_html(sample_data)}], return_only_outputs=True)
                    if batch_pred[0]['text'].strip() != sample['query']:
                        all_queries.append(batch_pred[0]['text'].strip())
                    llm_chain = LLMChain(llm=model, prompt=decompose_prompt, verbose=False)
                    batch_pred = llm_chain.batch([{"query": sample['query'], "table": TableFormat.format_html(sample_data)}], return_only_outputs=True)
                    all_queries.extend([q.strip() for q in batch_pred[0]['text'].split(';')])
                all_tokens += cb.total_tokens
                all_queries = list(set(all_queries))
                args_list = [{"query": q, "sample": sample, "k": sample_k} for q in all_queries]
                # print(len(args_list))
                ans_from_B = parallel_run_kwargs(scene_B, args_list)
                results = [res[0] for res in ans_from_B if 'Cannot get answer from sub-table' not in res[0]]
                all_tokens += sum([res[1] for res in ans_from_B])
                #With answer
                # results= []
                with get_openai_callback() as cb:
                    imp_input = scene_A(sample['query'], sample,sample_k,  False)
                    llm_chain = LLMChain(llm=model, prompt=get_k_shot_with_answer(), verbose=False)
                    batch_pred = llm_chain.batch([{"query":sample['query'],"SQL": imp_input[1], "table": imp_input[2], "information": '\n'.join(results)}], return_only_outputs=True)
                    # print(batch_pred[0])
                    all_tokens += cb.total_tokens
                    # print('ALL TOKENS', all_tokens)
                    ids.append(i)
                    labels.append(sample['query'])
                    outputs.append(batch_pred[0]['text'])
                    tokens.append(all_tokens)
                    extra_quries.append(';'.join(all_queries))
                    print(f'process {i}')
                    i += 1
                    
            #         if (i + 1) % 10 == 0:
            #                 print(f'saving {i}')

            #         i += 1
            #         pbar.update(1)
            # except ValueError as e:
            #     print(f'******************Value Error {i}****************************')
            #     i += 1
            #     pbar.update(1)
            # except BadRequestError as e:
            #     print('*************************Bad Request**************')
            #     i += 1
            #     pbar.update(1)
            except RateLimitError as e:
                print('*************************Rate limit**************')
                pass
            except ValueError as e:
                print(f'******************Value Error {i}****************************')
                i += 1
        
            except BadRequestError as e:
                print('*************************Bad Request**************')
                i += 1 

        pbar.update(1)
save_csv([outputs, labels, ids, tokens, extra_quries], ['preds', 'statements','ids', 'tokens', 'extra'], file_path=save_path)
    



Processing:   0%|                                                                                             …

process 684
process 754
process 757
process 761
process 762
process 1330


In [ ]:
len(outputs)

15

In [ ]:
save_csv([outputs, labels, ids, tokens, extra_quries], ['preds', 'statements','ids', 'tokens', 'extra'], file_path=save_path)

In [ ]:
save_csv([outputs, labels, ids], ['preds', 'statements','ids'], file_path=save_path)

In [ ]:
46: west berkshire brewery 's maggs magnificent mild be its most decorate beer between 1995 and 2009

SyntaxError: unterminated string literal (detected at line 1) (4231422701.py, line 1)

In [ ]:
20 ：Based on the information provided, the SQL query filtered for records where the place starts with 't9' and the country is 'united states'. The result of the query shows that there are 3 people who meet these criteria. However, the extra information states that there were actually 4 people who tied for ninth place, and all of them were from the United States.\n\nTherefore, the provided claim/query is false. The correct number of people from the United States who tied for ninth place is 4, not 3. \n\nFinal answer: 0
88: To verify whether the term start for Bashkim Fino is after the term start for Vilson Ahmeti, we need to compare the dates mentioned in the sub-table. From the sub-table:\n- Vilson Ahmeti's term started on December 10, 1991.\n- Bashkim Fino's term started on March 11, 1997.\n\nSince December comes before March in the calendar year, it is evident that Vilson Ahmeti's term started before Bashkim Fino's term. Therefore, the claim that the term start for Bashkim Fino is after the term start for Vilson Ahmeti is FALSE.\n\nHence, the answer is 0
72:To verify the claim that the gap between the first and last player being a total of 58.04 is true, we need to calculate the difference between the points of the first and last player based on the given information.\n\nGiven:\n- Total gap between the first and last rank is 18.\n- Rank of the first player is 1.\n- Rank of the last player is 19.\n\nLet's calculate the points difference between the first and last player:\npoints difference = (points of last player) - (points of first player)\n\nSince the total gap between the first and last rank is 18, we can calculate the points difference as follows:\npoints difference = 18 * (MAX(points) - MIN(points))\n\nGiven that the calculated point gap from the sub-table is 58.04, we can substitute this value into the formula:\n18 * 58.04 = 1044.72\n\nTherefore, the claim that the gap between the first and last player being a total of 58.04 is false.\n\nFinal answer: 0
76:The SQL query filters for competitors from France with a rank less than 5. Since the sub-table generated from the query shows no data, it means that there were no competitors from France who finished better than 5th place. \n\nTherefore, the provided claim that France's competitors all finished better than 5th place is TRUE.\n\nFinal answer: 1